## Parse sparql queries

In [ ]:
#  parse queries
# Install Node.js dependencies
!npm install sparqljs csv-parser csv-stringify

# Create the JavaScript file
js_code = """
const fs = require('fs');
const SparqlParser = require('sparqljs').Parser;
const csvParser = require('csv-parser');
const { stringify } = require('csv-stringify');

const parser = new SparqlParser();

async function executeQuery(query) {
  try {
    const parsedQuery = parser.parse(query);
    return JSON.stringify(parsedQuery);
  } catch (error) {
    console.error('Error parsing query:', query);
    return 'Error parsing the query.';
  }
}

async function writeBatchToCsv(batch, outputFile) {
  return new Promise((resolve, reject) => {
    const writeStream = fs.createWriteStream(outputFile, { flags: 'a' });
    const csvStringifier = stringify({ header: false, columns: [ 'Parsed_Query','Count'], delimiter: ',' });

    writeStream.on('error', (error) => {
      reject(error);
    });

    csvStringifier.pipe(writeStream);

    csvStringifier.on('end', () => {
      writeStream.end();
      resolve();
    });

    batch.forEach((entry) => {
      csvStringifier.write([ entry.Parsed_Query , entry.Count ]);
    });

    csvStringifier.end();
  });
}

async function main() {
  const inputCsvFile = 'query_prefixes_added.csv';
  const outputCsvFile = 'wikidata-robotic-parsed.csv';
  const batchSize = 1000000;

  const readStream = fs.createReadStream(inputCsvFile).pipe(csvParser());

  let batch = [];
  for await (const row of readStream) {
    const sparqlQuery = row['query'];
    const count = row['count'];

    const parsedQuery = await executeQuery(sparqlQuery);

    batch.push({ Parsed_Query: parsedQuery ,Count: count });

    if (batch.length >= batchSize) {
      await writeBatchToCsv(batch, outputCsvFile);
      console.log(`Processed ${batch.length} queries.`);
      batch = [];
    }
  }

  if (batch.length > 0) {
    await writeBatchToCsv(batch, outputCsvFile);
  }

  console.log('All SPARQL queries executed and results written to output CSV file.');
}

main();
"""

# Write JavaScript code to a file
with open("optimized.js", "w") as f:
    f.write(js_code)

# Python code to run the JavaScript script
python_code = """
import subprocess

def run_script():
    # Run the JavaScript code using Node.js with an increased memory limit
    result = subprocess.run(['node', '--max-old-space-size=30720', 'optimized.js'], stdout=subprocess.PIPE, text=True)

    # Print the output (JSON representation of the parsed SPARQL query)
    print(result.stdout)
    print('hi')

if __name__ == "__main__":
    run_script()
"""

# Write Python code to a file
with open("optimized_from_js_.py", "w") as f:
    f.write(python_code)

# Execute the script directly
print("Running the script...")

!python optimized_from_js_.py > parseoutput.txt 2>&1

# Print the log output
with open("parseoutput.txt", "r") as log_file:
    output = log_file.read()
    print(output)


##  We count triple patterns of parsed queries, to calculate complexity

In [ ]:
import pandas as pd

def count_subject_complexity(parsed_query):
    return parsed_query.count("subject")

# Replace 'input.csv' with the name of your input CSV file and 'output.csv' with the desired output CSV file name
input_file = 'parsed-output.csv'
output_file = 'parsed-output_complexity.csv

# Read the CSV file into a DataFrame with appropriate column names
column_names = ['query', 'parsed query']  # Adjust these column names as needed
df = pd.read_csv(input_file, header=None, names=column_names)

# Calculate complexity for each row and create a new column 'complexity'
df['complexity'] = df['parsed query'].apply(count_subject_complexity)

# Create a new DataFrame with 'query', 'parsed query', and 'complexity' columns
result_df = df[['query', 'parsed query', 'complexity']]

result_df = result_df.sort_values(by='complexity', ascending=False)

# Save the result DataFrame to a new CSV file with headers
result_df.to_csv(output_file, index=False)
print('sth')

## We calculate Informativness 

In [ ]:
import json
import csv

# Function to count variables and named nodes in the bgp section
def count_in_bgp(bgp_data):
    variable_count = 0
    named_node_count = 0

    if "triples" in bgp_data:
        triples = bgp_data["triples"]
        for triple in triples:
            subject_term_type = triple["subject"].get("termType", None)
            predicate_term_type = triple["predicate"].get("termType", None)
            object_term_type = triple["object"].get("termType", None)

            if subject_term_type == "Variable":
                variable_count += 1
            elif subject_term_type == "NamedNode":
                named_node_count += 1

            if predicate_term_type == "Variable":
                variable_count += 1
            elif predicate_term_type == "NamedNode":
                named_node_count += 1

            if object_term_type == "Variable":
                variable_count += 1
            elif object_term_type == "NamedNode":
                named_node_count += 1

    return variable_count, named_node_count

# Function to calculate informativeness and update CSV rows
def calculate_informativeness(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['Informativeness']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            parsed_query = row['Parsed_Query']

            try:
                data = json.loads(parsed_query)
            except json.JSONDecodeError:
                print(f"Error parsing JSON in row: {row}")
                continue

            total_variable_count = 0
            total_named_node_count = 0

            # Traverse through all "bgp" sections and count variables and named nodes in bgps
            def traverse(data):
                nonlocal total_variable_count, total_named_node_count
                if isinstance(data, dict):
                    if "type" in data and data["type"] == "bgp":
                        variable_count, named_node_count = count_in_bgp(data)
                        total_variable_count += variable_count
                        total_named_node_count += named_node_count
                    for key, value in data.items():
                        traverse(value)
                elif isinstance(data, list):
                    for item in data:
                        traverse(item)

            # Start traversing from the main JSON dictionary
            traverse(data)

            # Update the row with informativeness values
            if total_variable_count + total_named_node_count == 0:
                informativeness = 0.0  # Default value when denominator is zero
            else:
                informativeness = total_named_node_count / (total_variable_count + total_named_node_count)

            row['Informativeness'] = f'{informativeness}'
            writer.writerow(row)

# Provide the input and output file paths
input_csv_file = 'parsed-output.csv'
output_csv_file = 'informativness.csv'

# Call the function to calculate informativeness and update the output CSV file
calculate_informativeness(input_csv_file, output_csv_file)

## Find queries containing FILTER regex


In [ ]:
import pandas as pd

def queries_containing_FILTER(Query):
    return Query.count("FILTER regex")

# Replace 'input.csv' with the name of your input CSV file and 'output.csv' with the desired output CSV file name
input_file = 'query.csv'
output_file = 'query-with_filter.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

df['filterReg'] = df['Query'].apply(queries_containing_FILTER)



done


## Find queries containing OPTIONAL

In [ ]:
import pandas as pd

def count_optional(Query):
    return Query.count("OPTIONAL")

# Replace 'input.csv' with the name of your input CSV file and 'output.csv' with the desired output CSV file name
input_file = 'query.csv'
output_file = 'query-OPTIONAL.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Create a new column 'complexity' by counting the occurrences of 'subject' in 'parsed_query'
df['OPTIONAL'] = df['Query'].apply(count_optional)


done


## Find the queries with multiple prefixes(federation or high Interlinkness)

list of target prefixes:

DrugBank Vocabulary: http://bio2rdf.org/drugbank_vocabulary

PharmGKB Vocabulary: http://bio2rdf.org/pharmgkb_vocabulary

ClinicalTrials Vocabulary: http://bio2rdf.org/clinicaltrials_vocabulary

OMIM Vocabulary: http://bio2rdf.org/omim_vocabulary

MeSH Vocabulary: http://bio2rdf.org/mesh_vocabulary

Uniprot Vocabulary: http://bio2rdf.org/uniprot_vocabulary

Gene Ontology Vocabulary: http://bio2rdf.org/go_vocabulary

Reactome Vocabulary: http://bio2rdf.org/reactome_vocabulary

NCBI Taxonomy Vocabulary: http://bio2rdf.org/taxonomy_vocabulary

PubChem Vocabulary: http://bio2rdf.org/pubchem_vocabulary

PDB Vocabulary: http://bio2rdf.org/pdb_vocabulary

InterPro Vocabulary: http://bio2rdf.org/interpro_vocabulary

KEGG Vocabulary: http://bio2rdf.org/kegg_vocabulary

CHEBI Vocabulary: http://bio2rdf.org/chebi_vocabulary



['http://bio2rdf.org/pharmgkb_vocabulary:', 'http://bio2rdf.org/genatlas_vocabulary:', 'http://bio2rdf.org/chebi_vocabulary:', 'http://bio2rdf.org/chemspider_vocabulary:', 'http://bio2rdf.org/clinicaltrials_vocabulary:', 'http://bio2rdf.org/genbank_vocabulary:', 'http://bio2rdf.org/snomedct_vocabulary:', 'http://bio2rdf.org/ctd_vocabulary:', 'http://bio2rdf.org/dailymed_vocabulary:', 'http://bio2rdf.org/iuphar.ligand_vocabulary:', 'http://bio2rdf.org/ensembl_vocabulary:', 'http://bio2rdf.org/go_vocabulary:', 'http://bio2rdf.org/dbsnp_vocabulary:', 'http://bio2rdf.org/pubchem.substance_vocabulary:', 'http://bio2rdf.org/refseq_vocabulary:', 'http://bio2rdf.org/pubchem.compound_vocabulary:', 'http://bio2rdf.org/pdb_vocabulary:', 'http://bio2rdf.org/bindingdb_vocabulary:', 'http://bio2rdf.org/pubmed_vocabulary:', 'http://bio2rdf.org/huge_vocabulary:', 'http://bio2rdf.org/ccd_vocabulary:', 'http://bio2rdf.org/humancyc_vocabulary:', 'http://bio2rdf.org/atc_vocabulary:', 'http://bio2rdf.org/mesh_vocabulary:', 'http://bio2rdf.org/hgnc_vocabulary:', 'http://bio2rdf.org/ttd_vocabulary:', 'http://bio2rdf.org/ncbigene_vocabulary:', 'http://bio2rdf.org/uniprot_vocabulary:', 'http://bio2rdf.org/modbase_vocabulary:', 'http://bio2rdf.org/umls_vocabulary:', 'http://bio2rdf.org/drugbank_vocabulary:', 'http://bio2rdf.org/dpd_vocabulary:', 'http://bio2rdf.org/omim_vocabulary:', 'http://bio2rdf.org/kegg_vocabulary:', 'http://bio2rdf.org/mutdb_vocabulary:', 'http://bio2rdf.org/ndc_vocabulary:', 'http://bio2rdf.org/ndfrt_vocabulary:', 'http://bio2rdf.org/iuphar.receptor_vocabulary:', 'http://bio2rdf.org/hgnc.symbol_vocabulary:', 'http://bio2rdf.org/alfred_vocabulary:', 'http://bio2rdf.org/meddra_vocabulary:', 'http://bio2rdf.org/genecards_vocabulary:' ,'http://bio2rdf.org/interpro_vocabulary:', 'http://bio2rdf.org/reactome_vocabulary:', 'http://bio2rdf.org/mgi_vocabulary:']



## Interlinkness

In [ ]:
import pandas as pd

# Read the CSV file
input_file = 'query.csv'
output_file = 'query-interlinkness.csv'

keyword_list = ['http://bio2rdf.org/pharmgkb_vocabulary:', 'http://bio2rdf.org/genatlas_vocabulary:', 'http://bio2rdf.org/chebi_vocabulary:', 'http://bio2rdf.org/chemspider_vocabulary:', 'http://bio2rdf.org/clinicaltrials_vocabulary:', 'http://bio2rdf.org/genbank_vocabulary:', 'http://bio2rdf.org/snomedct_vocabulary:', 'http://bio2rdf.org/ctd_vocabulary:', 'http://bio2rdf.org/dailymed_vocabulary:', 'http://bio2rdf.org/iuphar.ligand_vocabulary:', 'http://bio2rdf.org/ensembl_vocabulary:', 'http://bio2rdf.org/go_vocabulary:', 'http://bio2rdf.org/dbsnp_vocabulary:', 'http://bio2rdf.org/pubchem.substance_vocabulary:', 'http://bio2rdf.org/refseq_vocabulary:', 'http://bio2rdf.org/pubchem.compound_vocabulary:', 'http://bio2rdf.org/pdb_vocabulary:', 'http://bio2rdf.org/bindingdb_vocabulary:', 'http://bio2rdf.org/pubmed_vocabulary:', 'http://bio2rdf.org/huge_vocabulary:', 'http://bio2rdf.org/ccd_vocabulary:', 'http://bio2rdf.org/humancyc_vocabulary:', 'http://bio2rdf.org/atc_vocabulary:', 'http://bio2rdf.org/mesh_vocabulary:', 'http://bio2rdf.org/hgnc_vocabulary:', 'http://bio2rdf.org/ttd_vocabulary:', 'http://bio2rdf.org/ncbigene_vocabulary:', 'http://bio2rdf.org/uniprot_vocabulary:', 'http://bio2rdf.org/modbase_vocabulary:', 'http://bio2rdf.org/umls_vocabulary:', 'http://bio2rdf.org/drugbank_vocabulary:', 'http://bio2rdf.org/dpd_vocabulary:', 'http://bio2rdf.org/omim_vocabulary:', 'http://bio2rdf.org/kegg_vocabulary:', 'http://bio2rdf.org/mutdb_vocabulary:', 'http://bio2rdf.org/ndc_vocabulary:', 'http://bio2rdf.org/ndfrt_vocabulary:', 'http://bio2rdf.org/iuphar.receptor_vocabulary:', 'http://bio2rdf.org/hgnc.symbol_vocabulary:', 'http://bio2rdf.org/alfred_vocabulary:', 'http://bio2rdf.org/meddra_vocabulary:', 'http://bio2rdf.org/genecards_vocabulary:' ,'http://bio2rdf.org/interpro_vocabulary:', 'http://bio2rdf.org/reactome_vocabulary:', 'http://bio2rdf.org/mgi_vocabulary:']

# Load the CSV into a pandas DataFrame
df = pd.read_csv(input_file)

# Function to count the number of keywords in a string
def count_keywords(query, keywords):
    return sum(1 for keyword in keywords if keyword in query)

# Apply the function to each row and create the 'Interlinkness' column
df['Interlinkness'] = df['Query'].apply(lambda x: count_keywords(x, keyword_list))


